## Python Imports
#### TODO: remove anything not used in the code

In [1]:
import keras
import librosa
import spotipy
import numpy as np

import src.vinyl.db_manager as crates
from src.obtain.spotify_metadata import generate_token, download_playlist_metadata
from src.vinyl.build_datasets import build_dataset

Using TensorFlow backend.


## [Globals](https://www.geeksforgeeks.org/global-local-variables-python/)

In [ ]:
# globals
spotify_username = 'djconxn'
user_id = "spotify:user:djconxn"

candidates_uri = 'spotify:playlist:69K5ogTF87NeSFvU9ePI3x'

#### TODO: add `features_dict` to model saving schema

In [2]:
model_save_path = "models/zouk_classifier_spectral_LSTM3.h5"

features_dict = {
    librosa.feature.mfcc : {'n_mfcc':12},
    librosa.feature.spectral_centroid : {},
    librosa.feature.chroma_stft : {'n_chroma':12},
    librosa.feature.spectral_contrast : {'n_bands':6},
    #librosa.feature.tempogram : {'win_length':192}
}

# Predict Zoukable Songs in Playlist

## User Input: Playlist Name + URI
Make sure to update the `playlist_name` and `playlist_uri` variables from the Spotify playlist to be screened.

In [3]:
playlist_name = "release_radar"
playlist_uri = "https://open.spotify.com/playlist/37i9dQZEVXbgM80lMZEYvm?si=K2cQ1BPUSTu22v_NGvtmNA"

In [4]:
token=generate_token(username=spotify_username)
sp = spotipy.Spotify(auth=token)

songs = crates.download_playlist_songs(sp, user_id, playlist_name, playlist_uri)

for song_id in songs:
    crates.get_preview_mp3(song_id)

playlist_data = build_dataset(songs, features_dict)

model = keras.models.load_model(model_save_path)

zoukable_preds = model.predict(playlist_data)

candidates = list(set(np.array(songs)[(zoukable_preds > 0.85).flatten()]))

zoukables = crates.get_playlist_songs("zoukables")
candidates = [x for x in candidates if x not in zoukables]

with open('zoukables_candidates.txt', 'w') as file:
    file.writelines('\n'.join(candidates))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Add Tracks to Candidates Playlist

In [8]:
# This can be run as a commandline utility from src/vinyl/add_songs_to_playlist.py

import pprint
import sys

import spotipy.util as util

'''
if len(sys.argv) > 3:
    user_id = sys.argv[1]
    candidates_uri = sys.argv[2]
    candidates = sys.argv[3:]
else:
    print "Usage: %s username playlist_id track_id ..." % (sys.argv[0],)
    sys.exit()
'''


scope = 'playlist-modify-public'
token = util.prompt_for_user_token(user_id, scope)

if token:
    sp = spotipy.Spotify(auth=token)
    sp.trace = False
    while candidates:
        end = min(100, len(candidates))
        results = sp.user_playlist_add_tracks(user_id, candidates_uri, candidates[:end])
        candidates = candidates[end:]
        print (results)
else:
    print ("Can't get token for", user_id)

Writing src/vinyl/add_songs_to_playlist.py
